# TensorFlow: Tabular Classify Binary

*Detecting Naval Mines with Binary Classification of Sonar Data.*

![mines](../../../images/banner/mines.png)

In [1]:
cd '~/Desktop/AIQC'

/Users/layne/Desktop/AIQC


---

## Example Data

In [2]:
from aiqc import datum
from aiqc.orm import Dataset

In [3]:
file_path = datum.get_path('sonar.csv')

This dataset is comprised of:

* *Features* = sonar readings that have been bounced off a distant object. 
* *Label* = either a rock or metal structure (potentially a naval mine).

Reference [Example Datasets](example_datasets.ipynb) for more information.

---

In [4]:
from aiqc.mlops import *

## Pipeline

Reference [High-Level API Docs](api_high_level.ipynb) for more information.

In [5]:
shared_dataset = Dataset.Tabular.from_path(file_path=file_path, file_format='csv')

In [ ]:
from sklearn.preprocessing import LabelBinarizer, PowerTransformer

In [ ]:
pipeline = Pipeline(
    Input(
        dataset = shared_dataset,
        encoders = Input.Encoder(
            PowerTransformer(method='yeo-johnson', copy=False),
            dtypes = ['float64']
    ),
        
    Target(
        dataset = shared_dataset,
        column = 'object',
        encoder = Target.Encoder(LabelBinarizer(sparse_output=False))
    ),

    Stratifier(
        size_test=0.12,
        size_validation=0.22
    )
)

---

## Modeling

Reference this great blog for machine learning cookbooks: [MachineLearningMastery.com "Binary Classification"](https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/).

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers as l

In [ ]:
def fn_build(features_shape, label_shape, **hp):
    m = tf.keras.models.Sequential()
    m.add(l.Input(shape=features_shape))
    m.add(l.Dense(hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    m.add(l.Dropout(0.30))
    m.add(l.Dense(hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    m.add(l.Dense(units=label_shape[0], activation='sigmoid', kernel_initializer='glorot_uniform'))
    return m

In [ ]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):
    model.compile(
        loss=loser
        , optimizer=optimizer
        , metrics=['accuracy']
    )
    model.fit(
        samples_train['features'], samples_train['labels']
        , validation_data = (samples_evaluate['features'], samples_evaluate['labels'])
        , verbose = 0
        , batch_size = 3
        , epochs = hp['epochs']
        , callbacks = [tf.keras.callbacks.History()]
    )
    return model

In [ ]:
hyperparameters = dict(
    neuron_count=[25, 50], 
    epochs=[75, 150]
)

In [ ]:
experiment = Experiment(
    Architecture(
        library = "keras",
        analysis_type = "classification_binary",
        fn_build = fn_build,
        fn_train = fn_train
    ),
    
    Trainer(
        pipeline = pipeline,
        repeat_count = 2,
        hyperparameters = hyperparameters   
    )
)

In [ ]:
experiment.run_jobs()

---

## Visualization & Interpretation

For more information on visualization of performance metrics, reference the [Dashboard](dashboard.html) documentation.